In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'/home/caleb/Cornell-Conversational-Analysis-Toolkit'

In [2]:
import convokit
import pickle
import pandas as pd


In [3]:
from convokit import Corpus, Conversation, Utterance

In [4]:
fake_news_dir = '/sauna/fake-news'
news_dir = '/sauna/reddit_201810_raw/corpus/newreddits_nsfw~-~news/news/'
donald_dir = '/sauna/reddit_201810_raw/corpus/TheTwoBeerQueers~-~The_Donald/The_Donald/'

In [5]:
fake_news_urls = {'breitbart.com', 'dcgazette.com', "dailymail.co.uk", 
                  "lewrockwell.com", "newswars.com", "gellerreport.com",
                  "dcclothesline.com", "thegatewaypundit.com", "trueactivist.com"
                 }

In [6]:
real_news_sites = ['BBC', 'Business Insider', 'Buzzfeed', 'CBS News', 'CNN', 'Daily Beast', 
                   'FT Westminster Blog', 'FiveThirtyEight', 'Fortune', 'Mercury News', 'NPR',
                   'National Review', 'New Yorker', 'Newsweek', 'PBS', 'Politico', 'Real Clear Politics', 'Reuters',
                   'Slate', 'The American Conservative', 'The Denver Post', 'The Guardian', 'The Hill', 'The New York Times',
                   'The Verge', 'USA Today', 'Vox', 'WSJ Washington Wire', 'Washington Monthly', 'Washington Post']

In [7]:
real_news_urls = {'bbc.com', 'businessinsider.com', 'buzzfeed.com', 'cbsnews.com', 'cnn.com', 'thedailybeast.com',
                  'ft.com', 'fivethirtyeight.com', 'fortune.com', 'mercurynews.com', 'npr.org', 'nationalreview.com',
                  'newyorker.com', 'newsweek.com', 'pbs.org', 'politico.com', 'realclearpol', 'reuters.com',
                  'slate.com', 'theamericanconservative.com', 'denverpost.com', 'theguardian.com', 'thehill.com',
                  'nytimes.com', 'theverge.com', 'usatoday.com', 'vox.com', 'wsj.com', 'washingtonmonthly.com', 
                  'washingtonpost.com'
                 }

In [9]:
donald_corpus = Corpus(filename=donald_dir)
# donald_corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_corpus_valid_convos'))

In [10]:
# Start time: 01 Oct 2016
# End time: 01 Oct 2018
start_time = 1475280000
end_time = 1538352000

In [11]:
from collections import defaultdict
real_news_convos = defaultdict(set)
fake_news_convos = defaultdict(set)
for convo in donald_corpus.iter_conversations():
    if start_time <= convo.meta['timestamp'] <= end_time:
        if convo.meta['domain'] in real_news_urls:
            real_news_convos[convo.meta['domain']].add(convo.id)
        elif convo.meta['domain'] in fake_news_urls:
            fake_news_convos[convo.meta['domain']].add(convo.id)

In [12]:
for k, v in real_news_convos.items():
    print("{}: {}".format(k, len(v)))
    
print("Total posts:", sum([len(v) for v in real_news_convos.values()]))

nytimes.com: 4071
cbsnews.com: 2079
thehill.com: 12938
theguardian.com: 3337
theverge.com: 383
wsj.com: 2047
cnn.com: 2835
nationalreview.com: 1403
vox.com: 510
npr.org: 1052
bbc.com: 3090
washingtonpost.com: 4781
politico.com: 3578
businessinsider.com: 2324
reuters.com: 5110
newsweek.com: 1078
usatoday.com: 2773
mercurynews.com: 342
buzzfeed.com: 673
newyorker.com: 301
thedailybeast.com: 1238
pbs.org: 224
slate.com: 479
fortune.com: 461
ft.com: 212
theamericanconservative.com: 264
denverpost.com: 205
fivethirtyeight.com: 236
washingtonmonthly.com: 21
Total posts: 58045


In [13]:
for k, v in fake_news_convos.items():
    print("{}: {}".format(k, len(v)))
    
print("Total posts:", sum([len(v) for v in fake_news_convos.values()]))

dailymail.co.uk: 13903
breitbart.com: 49274
thegatewaypundit.com: 16219
dcclothesline.com: 198
lewrockwell.com: 227
trueactivist.com: 28
dcgazette.com: 4
newswars.com: 392
gellerreport.com: 269
Total posts: 80514


In [14]:
from itertools import chain
real_news_convos_ids = list(chain.from_iterable(real_news_convos.values()))
fake_news_convos_ids = list(chain.from_iterable(fake_news_convos.values()))

In [15]:
valid_convos_ids = set(real_news_convos_ids + fake_news_convos_ids)

In [16]:
donald_corpus.print_summary_stats()

Number of Users: 617191
Number of Utterances: 38640598
Number of Conversations: 3830155


In [17]:
donald_corpus.filter_conversations_by(func=lambda convo: convo.id in valid_convos_ids)

In [18]:
donald_corpus.print_summary_stats()

Number of Users: 92476
Number of Utterances: 1274684
Number of Conversations: 138559


In [19]:
donald_corpus.dump('donald_corpus_valid_convos_2year', base_path=fake_news_dir)

In [20]:
thread_pfxs = donald_corpus.utterance_threads(prefix_len=8, include_root=False)
convos_with_valid_len = set()
convo_to_thread = defaultdict(list)
for thread_id, utts in thread_pfxs.items():
    if len(utts) >= 8:
        convos_with_valid_len.add(utts[next(iter(utts))].root)
        convo_to_thread[utts[next(iter(utts))].root].append(thread_id)

In [21]:
from random import choice

for convo, threads in convo_to_thread.items():
    convo_to_thread[convo] = choice(threads)

In [28]:
thread_to_convo = {v: k for k, v in convo_to_thread.items() if len(v) > 0}

In [22]:
real_news_convos_ids[:10]

['6oiwcb',
 '5wmjg3',
 '5u4uxc',
 '86i3r1',
 '6i2dzz',
 '5mroiq',
 '56swim',
 '5by9qm',
 '65c0f0',
 '6kilc8']

In [23]:
real_news_thread_ids = [convo_to_thread[convo_id] for convo_id in real_news_convos_ids if len(convo_to_thread[convo_id]) > 0]
fake_news_thread_ids = [convo_to_thread[convo_id] for convo_id in fake_news_convos_ids if len(convo_to_thread[convo_id]) > 0]

In [24]:
len(real_news_thread_ids)

2241

In [25]:
len(fake_news_thread_ids)

4333

In [26]:
from random import sample
fake_news_thread_sample = sample(fake_news_thread_ids, len(real_news_thread_ids))

In [29]:
fake_news_convos_sample = [thread_to_convo[thread_id] for thread_id in fake_news_thread_sample]

In [30]:
real_news_long_convos = [thread_to_convo[thread_id] for thread_id in real_news_thread_ids]

In [31]:
paired_convos = set(fake_news_convos_sample).union(real_news_long_convos)

In [32]:
balanced_threads = real_news_thread_ids + fake_news_thread_sample

In [33]:
len(set(balanced_threads))

4482

In [35]:
donald_corpus.filter_conversations_by(func=lambda convo: convo.id in paired_convos)

In [36]:
donald_corpus.dump('donald_corpus_paired_2year', base_path=fake_news_dir)

In [ ]:
# donald_corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_corpus_paired'))

In [37]:
donald_corpus.print_summary_stats()

Number of Users: 62295
Number of Utterances: 496334
Number of Conversations: 4482


In [38]:
hc = convokit.HyperConvo(min_thread_len=8, prefix_len=8, include_root=False)

In [39]:
motif_feats = hc.retrieve_motif_counts(donald_corpus)

In [40]:
balanced_threads = set(balanced_threads)

In [41]:
motif_feats = {k: v for k, v in motif_feats.items() if k in balanced_threads}

In [42]:
motif_counts_df = pd.DataFrame.from_dict(motif_feats, orient='index')
motif_feat_names = list(motif_counts_df.columns)

In [43]:
hyperconv_feats = hc.retrieve_feats(donald_corpus)
hyperconv_feats = {k: v for k, v in hyperconv_feats.items() if k in balanced_threads}

/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:182: RuntimeWarning: invalid value encountered in long_scalars
  "norm.max": lambda l: np.max(l) / np.sum(l),
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:187: RuntimeWarning: invalid value encountered in long_scalars
  if len(l) > 1 else np.nan,
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2614: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=0)
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:194: Runtim

In [44]:
hyperconv_df = pd.DataFrame.from_dict(hyperconv_feats, orient='index')
hyperconv_feat_names = list(hyperconv_df.columns)

In [45]:
path_feats = hc.retrieve_motif_pathway_stats(donald_corpus)
path_feats = {k: v for k, v in path_feats.items() if k in balanced_threads}

In [46]:
path_stats_df = pd.DataFrame.from_dict(path_feats, orient='index')
columns = ['PATH-'+', '.join(filter(lambda x: type(x) == str, col)).strip() for col in path_stats_df.columns.values]
path_stats_df.columns = columns
path_feat_names = list(path_stats_df.columns)

In [47]:
feats_df = pd.concat([hyperconv_df, path_stats_df, motif_counts_df], axis=1)

In [48]:
fake_news_thread_sample = set(fake_news_thread_sample)
y = [int(thread_id in fake_news_thread_sample) for thread_id in list(feats_df.index)]

In [49]:
import numpy as np
y = np.array(y)

In [50]:
feats_df['y'] = y

In [51]:
feats_df.shape

(4482, 261)

In [52]:
feats_df_no_na = feats_df.dropna()

In [53]:
len(feats_df_no_na)

4443

In [54]:
sum(feats_df_no_na['y'])

2224

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


for feature_set, name in [(hyperconv_feat_names, "hyperconvo"),
                        (motif_feat_names, "motifs"),
                        (path_feat_names, "motifpaths"),
                        (hyperconv_feat_names + motif_feat_names, "hyperconv-motif"),
                        (hyperconv_feat_names + path_feat_names, "hyperconv-paths"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names, "hyperconvo-motifall"),
                       ]:
    clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      
    loo = LeaveOneOut()

    X = np.array(feats_df_no_na[feature_set])
    y = np.array(feats_df_no_na['y'])
    scores = cross_val_score(clf, X, y, cv=loo)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     clf.fit(X_train, y_train)
#     train_score = clf.score(X_train, y_train)
#     test_score = clf.score(X_test, y_test)
#     print("- {}, train: {:.4f}, test: {:.4f}".format(name, train_score, test_score))
    print("- {}, cv_accuracy: {:.4f}".format(name, scores.mean()))

- hyperconvo, cv_accuracy: 0.5192
- motifs, cv_accuracy: 0.5226
- motifpaths, cv_accuracy: 0.5123
- hyperconv-motif, cv_accuracy: 0.5138
- hyperconv-paths, cv_accuracy: 0.5123


In [58]:
thread_pfxs = donald_corpus.utterance_threads(prefix_len=8, include_root=False)

In [110]:
from sklearn.model_selection import LeaveOneOut 
from sklearn.model_selection import cross_val_score




## BoW prediction

In [59]:
thread_pfx_texts = {k: " ".join([utt.text for utt in v.values()]) for k, v in thread_pfxs.items() if k in balanced_threads}

In [60]:
y = [int(thread_id in fake_news_thread_sample) for thread_id in list(feats_df.index)]

In [61]:
train_ids, test_ids, y_train, y_test = train_test_split(list(balanced_threads), y, test_size=0.2, random_state=42)

In [99]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0.05, max_df=0.7, ngram_range=(1,3))

In [100]:
X_train = cv.fit_transform([thread_pfx_texts[train_id] for train_id in train_ids]).toarray()

In [101]:
X_test = cv.transform([thread_pfx_texts[test_id] for test_id in test_ids]).toarray()

In [102]:
clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      

clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)
print("- {}, train: {:.4f}, test: {:.4f}".format("BoW", train_score, test_score))


- BoW, train: 0.6485, test: 0.5162


In [103]:
def print_extreme_coefs(clf, feature_names, num_features: int = 5):
    coefs = clf.named_steps['logreg'].coef_[0].tolist()

    assert len(feature_names) == len(coefs)

    feats_coefs = sorted(list(zip(feature_names, coefs)), key=lambda x: x[1], reverse=True)

    print()
    print("TOP {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[:num_features]:
        print("{}: {:.3f}".format(ft, coef))
    print()
    print("BOTTOM {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[-num_features:]:
        print("{}: {:.3f}".format(ft, coef))
    print()

In [104]:
print_extreme_coefs(clf, cv.get_feature_names(), num_features = 10)


TOP 10 FEATURES
you re: 0.319
they re: 0.296
able: 0.286
trying: 0.223
wouldn: 0.170
hillary: 0.164
we re: 0.163
at least: 0.162
with: 0.151
first: 0.146

BOTTOM 10 FEATURES
going to: -0.137
have the: -0.137
least: -0.143
be: -0.152
bad: -0.156
with the: -0.156
trying to: -0.225
able to: -0.293
clinton: -0.472
re: -0.482

